# Model Hyperparameter Optimization Notebook

This notebook performs hyperparameter tuning on the best performing model from the training phase.

The notebook performs the following steps:
1. Load configuration and best performing model
2. Load processed training and test data
3. Define hyperparameter grids from config
4. Initialize HyperparameterOptimizer
5. Run hyperparameter optimization (limited to 2 hours max)
6. Evaluate optimized model on test set
7. Verify F1-score improvement meets minimum threshold of 0.80
8. Save optimized model as best_model.pkl

## 1. Setup and Imports

In [ ]:
import sys
import os
import pickle
import yaml
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Add src to path
sys.path.append('..')

# Import custom modules
from src.modeling.hyperparameter_optimizer import HyperparameterOptimizer
from src.modeling.model_registry import ModelRegistry
from src.evaluation.model_evaluator import ModelEvaluator
from src.utils.logger import get_logger

# Set up logger
logger = get_logger(__name__)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('All imports successful!')

## 2. Load Configuration

In [ ]:
# Load configuration from YAML file
config_path = '../config/config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

print('Configuration loaded successfully!')
print(f"\nHyperparameter tuning enabled: {config['hyperparameter_tuning']['enabled']}")
print(f"Search method: {config['hyperparameter_tuning']['method']}")
print(f"CV folds: {config['hyperparameter_tuning']['cv_folds']}")
print(f"Number of iterations: {config['hyperparameter_tuning']['n_iter']}")
print(f"Primary metric: {config['evaluation']['primary_metric']}")

## 3. Load Processed Data

In [ ]:
# Load processed training and test data
processed_data_path = config['data']['processed_data_path']

print(f'Loading data from {processed_data_path}...')

# Load datasets
with open(f'{processed_data_path}X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open(f'{processed_data_path}X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

with open(f'{processed_data_path}y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open(f'{processed_data_path}y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

print('\nData loaded successfully!')
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_test shape: {y_test.shape}')

## 4. Load Best Performing Model from Previous Training

In [ ]:
# Initialize ModelRegistry
registry = ModelRegistry(models_dir='../models')

# List all available models
print('Available models in registry:')
print('=' * 80)
models_list = registry.list_models()
for model_info in models_list:
    print(f"Model: {model_info['model_name']} v{model_info['version']}")
    if 'metrics' in model_info and model_info['metrics']:
        print(f"  F1-Score: {model_info['metrics'].get('f1_score', 'N/A')}")
        print(f"  Accuracy: {model_info['metrics'].get('accuracy', 'N/A')}")
    print()

# Load the best model based on F1-score
print('\nLoading best performing model...')
best_model_result = registry.get_best_model(metric='f1_score')

if best_model_result is None:
    raise ValueError("No models found in registry. Please run the training notebook first.")

best_model, best_metadata = best_model_result

print('\n' + '=' * 80)
print('Best Model Loaded:')
print('=' * 80)
print(f"Model Name: {best_metadata.get('model_name', 'Unknown')}")
print(f"Model Type: {best_metadata.get('model_type', 'Unknown')}")
print(f"Version: {best_metadata.get('version', 'Unknown')}")
print(f"Training Date: {best_metadata.get('training_date', 'Unknown')}")
print(f"\nCurrent Performance Metrics:")
if 'metrics' in best_metadata:
    for metric, value in best_metadata['metrics'].items():
        if value is not None:
            print(f"  {metric}: {value:.4f}")
print('=' * 80)

## 5. Define Hyperparameter Grids from Config

In [ ]:
# Determine which model to optimize based on loaded best model
model_type = best_metadata.get('model_type', '')
original_model_name = best_metadata.get('model_name', '')

print(f'Model type to optimize: {model_type}')
print(f'Original model name: {original_model_name}')

# Get hyperparameter grid from config
param_grids = config['hyperparameter_tuning']['param_grids']

# Determine which parameter grid to use
if 'RandomForest' in model_type or 'random_forest' in original_model_name:
    param_grid = param_grids['random_forest']
    base_model = RandomForestClassifier(random_state=config['data']['random_state'])
    model_name_for_optimization = 'random_forest'
    print('\nUsing Random Forest hyperparameter grid')
elif 'XGB' in model_type or 'xgboost' in original_model_name:
    param_grid = param_grids['xgboost']
    base_model = XGBClassifier(random_state=config['data']['random_state'], eval_metric='logloss')
    model_name_for_optimization = 'xgboost'
    print('\nUsing XGBoost hyperparameter grid')
else:
    # Default to random forest if model type is unclear
    print(f'\nWarning: Could not determine model type from {model_type}. Defaulting to Random Forest.')
    param_grid = param_grids['random_forest']
    base_model = RandomForestClassifier(random_state=config['data']['random_state'])
    model_name_for_optimization = 'random_forest'

print('\nHyperparameter Grid:')
print('=' * 80)
for param, values in param_grid.items():
    print(f"  {param}: {values}")
print('=' * 80)

## 6. Initialize HyperparameterOptimizer

In [ ]:
# Initialize HyperparameterOptimizer with configuration
optimizer = HyperparameterOptimizer(
    param_grid=param_grid,
    search_method=config['hyperparameter_tuning']['method'],
    cv_folds=config['hyperparameter_tuning']['cv_folds'],
    n_iter=config['hyperparameter_tuning']['n_iter'],
    scoring='f1_weighted',
    n_jobs=-1
)

print('HyperparameterOptimizer initialized successfully!')
print(f"Search method: {config['hyperparameter_tuning']['method']}")
print(f"CV folds: {config['hyperparameter_tuning']['cv_folds']}")
print(f"Scoring metric: f1_weighted")

## 7. Run Hyperparameter Optimization (Limited to 2 Hours)

In [ ]:
# Record start time
optimization_start_time = time.time()
max_optimization_time = 2 * 60 * 60  # 2 hours in seconds

print('Starting hyperparameter optimization...')
print('=' * 80)
print(f'Maximum optimization time: 2 hours')
print(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
print('=' * 80)
print('\nThis may take a while. Progress will be displayed below...\n')

# Run optimization
search_result = optimizer.optimize(
    model=base_model,
    X_train=X_train,
    y_train=y_train,
    verbose=2
)

# Calculate optimization time
optimization_time = time.time() - optimization_start_time
optimization_hours = optimization_time / 3600

print('\n' + '=' * 80)
print('Hyperparameter Optimization Completed!')
print('=' * 80)
print(f'Total optimization time: {optimization_hours:.2f} hours ({optimization_time:.2f} seconds)')
print(f'End time: {time.strftime("%Y-%m-%d %H:%M:%S")}')

# Check if optimization exceeded time limit
if optimization_time > max_optimization_time:
    print(f'\n⚠️  Warning: Optimization exceeded 2-hour limit by {(optimization_time - max_optimization_time)/60:.2f} minutes')
else:
    print(f'\n✓ Optimization completed within time limit')

print('=' * 80)

## 8. Display Optimization Results

In [ ]:
# Get best parameters and score
best_params = optimizer.get_best_params()
best_cv_score = optimizer.get_best_score()
optimized_model = optimizer.get_best_estimator()

print('\n' + '=' * 80)
print('OPTIMIZATION RESULTS')
print('=' * 80)

print(f'\nBest Cross-Validation F1-Score: {best_cv_score:.4f}')

print('\nBest Hyperparameters:')
for param, value in best_params.items():
    print(f"  {param}: {value}")

# Compare with original model performance
if 'metrics' in best_metadata and 'f1_score' in best_metadata['metrics']:
    original_f1 = best_metadata['metrics']['f1_score']
    improvement = best_cv_score - original_f1
    improvement_pct = (improvement / original_f1) * 100
    
    print(f'\nComparison with Original Model:')
    print(f"  Original F1-Score: {original_f1:.4f}")
    print(f"  Optimized CV F1-Score: {best_cv_score:.4f}")
    print(f"  Improvement: {improvement:+.4f} ({improvement_pct:+.2f}%)")

print('=' * 80)

## 9. Visualize Optimization Results

In [ ]:
# Get CV results
cv_results = optimizer.get_cv_results()

# Create visualization of parameter search
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Score distribution
ax1 = axes[0]
scores = cv_results['mean_test_score']
ax1.hist(scores, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(best_cv_score, color='red', linestyle='--', linewidth=2, label=f'Best Score: {best_cv_score:.4f}')
ax1.set_xlabel('F1-Score', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Cross-Validation Scores', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(axis='y', alpha=0.3)

# Plot 2: Top 10 parameter combinations
ax2 = axes[1]
top_n = min(10, len(scores))
top_indices = scores.argsort()[-top_n:][::-1]
top_scores = scores[top_indices]
top_ranks = range(1, top_n + 1)

bars = ax2.barh(top_ranks, top_scores, color='lightcoral', edgecolor='black', alpha=0.7)
ax2.set_xlabel('F1-Score', fontsize=12)
ax2.set_ylabel('Rank', fontsize=12)
ax2.set_title(f'Top {top_n} Parameter Combinations', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

# Add score labels
for i, (rank, score) in enumerate(zip(top_ranks, top_scores)):
    ax2.text(score + 0.001, rank, f'{score:.4f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('../reports/figures/hyperparameter_optimization_results.png', dpi=300, bbox_inches='tight')
plt.show()

print('Optimization visualization saved!')

## 10. Evaluate Optimized Model on Test Set

In [ ]:
# Initialize ModelEvaluator
evaluator = ModelEvaluator(output_dir='../reports/figures')

print('Evaluating optimized model on test set...')
print('=' * 80)

# Evaluate optimized model
optimized_metrics = evaluator.evaluate_model(
    model=optimized_model,
    X_test=X_test,
    y_test=y_test,
    model_name=f'{model_name_for_optimization}_optimized'
)

# Plot confusion matrix
evaluator.plot_confusion_matrix_heatmap(
    y_test,
    optimized_model.predict(X_test),
    model_name=f'{model_name_for_optimization}_optimized'
)

print('\n' + '=' * 80)
print('OPTIMIZED MODEL TEST SET PERFORMANCE')
print('=' * 80)

# Display metrics
metrics_clean = {k: v for k, v in optimized_metrics.items() if k != 'confusion_matrix'}
for metric, value in metrics_clean.items():
    if value is not None:
        print(f"  {metric}: {value:.4f}")

print('=' * 80)

## 11. Verify F1-Score Improvement and Threshold

In [ ]:
# Define minimum F1-score threshold
min_f1_threshold = 0.80
optimized_f1 = optimized_metrics['f1_score']

print('\n' + '=' * 80)
print('F1-SCORE VERIFICATION')
print('=' * 80)

print(f'\nMinimum F1-Score Threshold: {min_f1_threshold:.2f}')
print(f'Optimized Model F1-Score: {optimized_f1:.4f}')

# Check if threshold is met
if optimized_f1 >= min_f1_threshold:
    print(f'\n✅ SUCCESS: F1-score meets minimum threshold!')
    print(f'   Exceeds threshold by: {(optimized_f1 - min_f1_threshold):.4f}')
    threshold_met = True
else:
    print(f'\n⚠️  WARNING: F1-score below minimum threshold')
    print(f'   Falls short by: {(min_f1_threshold - optimized_f1):.4f}')
    threshold_met = False

# Compare with original model
if 'metrics' in best_metadata and 'f1_score' in best_metadata['metrics']:
    original_f1 = best_metadata['metrics']['f1_score']
    f1_improvement = optimized_f1 - original_f1
    f1_improvement_pct = (f1_improvement / original_f1) * 100
    
    print(f'\nComparison with Original Model:')
    print(f"  Original Test F1-Score: {original_f1:.4f}")
    print(f"  Optimized Test F1-Score: {optimized_f1:.4f}")
    print(f"  Improvement: {f1_improvement:+.4f} ({f1_improvement_pct:+.2f}%)")
    
    if f1_improvement > 0:
        print(f'\n✅ Model performance improved after optimization!')
    elif f1_improvement == 0:
        print(f'\n➡️  Model performance unchanged after optimization')
    else:
        print(f'\n⚠️  Model performance decreased after optimization')

print('=' * 80)

## 12. Save Optimized Model as best_model.pkl

In [ ]:
# Prepare metadata for optimized model
optimized_metadata = {
    'model_name': 'best_model',
    'version': '2.0.0',
    'original_model_name': model_name_for_optimization,
    'optimization_method': config['hyperparameter_tuning']['method'],
    'cv_folds': config['hyperparameter_tuning']['cv_folds'],
    'optimization_time_hours': optimization_hours,
    'best_cv_score': best_cv_score,
    'threshold_met': threshold_met,
    'min_f1_threshold': min_f1_threshold
}

# Clean metrics (remove confusion matrix)
metrics_to_save = {k: v for k, v in optimized_metrics.items() if k != 'confusion_matrix'}

# Save optimized model
print('\nSaving optimized model...')
print('=' * 80)

optimized_model_path = registry.save_model(
    model=optimized_model,
    model_name='best_model',
    version='2.0.0',
    metrics=metrics_to_save,
    hyperparameters=best_params,
    additional_metadata=optimized_metadata
)

print(f'\n✅ Optimized model saved successfully!')
print(f'   Path: {optimized_model_path}')
print('=' * 80)

## 13. Summary and Conclusions

In [ ]:
print('\n' + '=' * 100)
print('HYPERPARAMETER OPTIMIZATION SUMMARY')
print('=' * 100)

print(f'\n📊 Optimization Details:')
print(f'   - Model Type: {model_type}')
print(f'   - Search Method: {config["hyperparameter_tuning"]["method"]}')
print(f'   - CV Folds: {config["hyperparameter_tuning"]["cv_folds"]}')
print(f'   - Iterations: {config["hyperparameter_tuning"]["n_iter"]}')
print(f'   - Optimization Time: {optimization_hours:.2f} hours')

print(f'\n🎯 Performance Results:')
print(f'   - Best CV F1-Score: {best_cv_score:.4f}')
print(f'   - Test Set Accuracy: {optimized_metrics["accuracy"]:.4f}')
print(f'   - Test Set Precision: {optimized_metrics["precision"]:.4f}')
print(f'   - Test Set Recall: {optimized_metrics["recall"]:.4f}')
print(f'   - Test Set F1-Score: {optimized_metrics["f1_score"]:.4f}')
if optimized_metrics['roc_auc'] is not None:
    print(f'   - Test Set ROC-AUC: {optimized_metrics["roc_auc"]:.4f}')

print(f'\n🔧 Best Hyperparameters:')
for param, value in best_params.items():
    print(f'   - {param}: {value}')

print(f'\n✅ Requirements Verification:')
print(f'   - Requirement 6.1: Hyperparameter tuning performed ✓')
print(f'   - Requirement 6.2: Multiple hyperparameters optimized ✓')
print(f'   - Requirement 6.3: Cross-validation used during search ✓')
print(f'   - Requirement 6.4: Optimization completed within 2 hours: {"✓" if optimization_time <= max_optimization_time else "✗"}')
print(f'   - Requirement 6.5: F1-score threshold (0.80) met: {"✓" if threshold_met else "✗"}')

if threshold_met:
    print(f'\n🎉 SUCCESS: All requirements met!')
    print(f'   The optimized model achieves F1-score of {optimized_f1:.4f}, exceeding the minimum threshold of {min_f1_threshold:.2f}')
else:
    print(f'\n⚠️  Note: F1-score threshold not met')
    print(f'   Current F1-score: {optimized_f1:.4f}')
    print(f'   Required threshold: {min_f1_threshold:.2f}')
    print(f'   Consider: Collecting more data, feature engineering, or trying different models')

print(f'\n💾 Model Saved:')
print(f'   - File: best_model_v2.0.0.pkl')
print(f'   - Location: {optimized_model_path}')

print('\n' + '=' * 100)
print('Hyperparameter optimization notebook completed successfully!')
print('=' * 100)